# In this notebook we look into the performance of semi-dual damped Newton in the numpy framework.

In [ ]:
from __future__ import division
import os
import numpy as np
import time
import matplotlib.pyplot as plt                                                                              
import warnings
warnings.filterwarnings( 'ignore' )
np.random.seed(1234)
import computational_OT
%matplotlib inline
%load_ext autoreload
%autoreload 2

In [ ]:
relative_path_to_new_folder = "../Images"
os.makedirs( relative_path_to_new_folder, exist_ok = True )
if not os.path.isdir( "../Images/SemiDual_dampedNewton_images_numpy" ):
    os.makedirs( "../Images/SemiDual_dampedNewton_images_numpy" )

In [ ]:
"""To compute distance matrix"""
def distmat( x, y ):
    return np.sum( x ** 2, 0 )[:,None] + np.sum( y ** 2, 0 )[None,:] - 2 * x.transpose().dot( y )

"""To Normalise a vector"""
normalize = lambda a: a/np.sum( a )

"""To Compute P"""
def GetP( u, K, v ):
    return u[:,None] * K * v[None,:]

def plotp( x, col, plt, scale = 200, edgecolors = "k" ):
  return plt.scatter( x[0,:], x[1,:], s = scale, edgecolors = edgecolors, c = col, cmap = 'plasma', linewidths = 2 )

In [ ]:
def generate_data( N ):
    """
     N is a list of the size of the data on x and y
    """
    x = np.random.rand( 2, N[0] ) - 0.5
    theta = 2 * np.pi * np.random.rand( 1, N[1] )
    r = 0.8 + 0.2 * np.random.rand( 1, N[1] )
    y = np.vstack( ( r * np.cos( theta ), r * np.sin( theta ) ) )
    return x, y

In [ ]:
N = [ 1000, 1100 ]
x, y = generate_data( N )

\## Entropy regularized dual-formulation
The dual formulation of the entropy regularized OT is given by:
$$
OT_{\varepsilon}(\alpha,\beta) = \max_{f\in \mathbb{R}^{n}, g\in\mathbb{R}^{m}} \langle f, \alpha \rangle + \langle g, \beta \rangle - \varepsilon\left(\langle\alpha \otimes \beta, e^{\frac{f}{\varepsilon}}\odot K \odot e^{\frac{g}{\varepsilon}}  \rangle-1\right)\ ,
$$
where
$$
\alpha \in \mathcal{M}_{1}(\mathcal{X}),\ \beta \in \mathcal{M}_{1}(\mathcal{Y}),\ \varepsilon>0,\ f\in\mathbb{R}^{n},\ g\in \mathbb{R}^{m}\ .
$$

Let us denote the objective function of the above formulation by $Q^{semi}_{\alpha, \beta, \varepsilon}(f)$.

# The semi-dual formulation of OT is given by:
Using the Shrodinger-bridge equations between the potentials, that is, $g_{j} = -\varepsilon\log\left(\sum_{i}\exp\left(\frac{f_{i}-C_{ij}}{\varepsilon}\right)\alpha_{i}\right)\ , \ \forall j = 1,\dots,m$, the dual formulation of the objective function $Q_{\alpha, \beta,\varepsilon}$ reduces to the semi-dual formulation of the objective function given by,
$$
Q^{semi}_{\alpha, \beta, \varepsilon}(f) = \langle f, \alpha \rangle + \langle g(f,C,\varepsilon), \beta \rangle\ , 
$$
where
$g(f,C,\varepsilon)_{j} = -\varepsilon\log\left(\sum_{i}\exp\left(\frac{f_{i}-C_{ij}}{\varepsilon}\right)\alpha_{i}\right)$.

In this setup, the gradients and the Hessian is as follows,

$a)$ Gradients:
$$
\nabla_{f}Q^{semi}_{\alpha, \beta, \varepsilon}(f)_{i} = \frac{1}{\varepsilon}\alpha_{i}\left(1-\sum_{s=1}^{n}\frac{e^{\frac{f_{i}-C_{ij}}{\varepsilon}}\beta_{s}}{\left(\sum_{t=1}^{n}\alpha_{t}e^{\frac{f_{t}-C_{ts}}{\varepsilon}}\right)}\right)\ ,\ \forall i = 1,\dots,n\ .
$$

b) Hessian:
$$
\nabla^{2}_{f}Q^{semi}_{\alpha, \beta, \varepsilon}(f)_{ii} = \frac{-1}{\varepsilon}\sum_{s=1}^{m}\left(\alpha_{i}\exp\left(\frac{f_{i}+g(f,C,\varepsilon)_{s}-C_{is}}{\varepsilon}\right)\right)\left(1 - \alpha_{i}\left(\exp\left(\frac{f_{i}+g(f,C,\varepsilon)_{s}-C_{is}}{\varepsilon}\right)\right)\right)\beta_{s}\ ,\ \forall i =1,\dots,n
$$
and
$$
\nabla^{2}_{f}Q^{semi}_{\alpha, \beta, \varepsilon}(f)_{ij} = \frac{1}{\varepsilon}\sum_{s=1}^{m}\alpha_{i}\alpha_{j}\left(\exp\left(\frac{f_{i}+g(f,C,\varepsilon)_{s}-C_{is}}{\varepsilon}\right)\right)\left(\exp\left(\frac{f_{j}+g(f,C,\varepsilon)_{s}-C_{js}}{\varepsilon}\right)\right)\beta_{s}\ ,\ \forall i \neq j = 1,\dots,n\ .
$$
Now we plug-in these gradients and Hessian in damped Newton algorithm as we did before.

Here we also incorporate the exp-log stabilization to stabilize $g$,the gradients and the Hessian in the following way,
$$
f^{C}_{j} \leftarrow \min_{i}(C_{ij}-f_{i})\ ,  \ \forall j = 1,\dots,m,\ \text{the C-transform of f}\ . 
$$
$$
g_{j} = f^{C}_{j} -\varepsilon\log\left(\sum_{i}\exp\left(\frac{f_{i}+f^{C}_{j}-C_{ij}}{\varepsilon}\right)\alpha_{i}\right)\ ,  \ \forall j = 1,\dots,m\ ,
$$
$$
\nabla_{f}Q^{semi}_{\alpha, \beta, \varepsilon}(f)_{i} = \frac{1}{\varepsilon}\alpha_{i}\left(1-\sum_{s=1}^{n}\frac{e^{\frac{f_{i}+f^{C}_{j}-C_{ij}}{\varepsilon}}\beta_{s}}{\left(\sum_{t=1}^{n}\alpha_{t}e^{\frac{f_{t}+f^{C}_{j}-C_{ts}}{\varepsilon}}\right)}\right)\ ,\ \forall i = 1,\dots,n\ , 
$$
$$
\nabla^{2}_{f}Q^{semi}_{\alpha, \beta, \varepsilon}(f)_{ii} = \frac{-1}{\varepsilon}\sum_{s=1}^{m}\left(\alpha_{i}\exp\left(\frac{f_{i}+g(f,C,\varepsilon)_{s}-C_{is}}{\varepsilon}\right)\right)\left(1 - \alpha_{i}\left(\exp\left(\frac{f_{i}+g(f,C,\varepsilon)_{s}-C_{is}}{\varepsilon}\right)\right)\right)\beta_{s}\ ,\ \forall i =1,\dots,n\ , 
$$
$$
\nabla^{2}_{f}Q^{semi}_{\alpha, \beta, \varepsilon}(f)_{ij} = \frac{1}{\varepsilon}\sum_{s=1}^{m}\alpha_{i}\alpha_{j}\left(\exp\left(\frac{f_{i}+g(f,C,\varepsilon)_{s}-C_{is}}{\varepsilon}\right)\right)\left(\exp\left(\frac{f_{j}+g(f,C,\varepsilon)_{s}-C_{js}}{\varepsilon}\right)\right)\beta_{s}\ ,\ \forall i \neq j = 1,\dots,n\ .
$$

# I. Semidual damped Newton (Direct inversion / No preconditioning)

## Experiment

In [ ]:
print( " Semi dual damped Newton... " )
print( " Doing for (",N[0], N[1],"). " )
withoutprecond_epsilons = [ 0.5, 0.1, 0.05, 0.03, 0.02 ]                                                                                                                                                                                                                                                                                                                                                    
rho = 0.7
c = 0.1
Semi_dual_dampedNewtonP = []    
results_semi_dual_dampedNewton = []
times_semi_dual_dampedNewton = []
Hessians_semi_dual_dampedNewton = []
#Cost matrix
C = distmat( x, y )
# a and b
a = normalize( np.ones( N[0] ) )
b = normalize( np.ones( N[1] ) )
for eps in withoutprecond_epsilons:
    K = np.exp( - C/eps )
    print( " For epsilon = " + str(eps) + ":" )   
    f = a
    print( " |-  Iterating" )  
    start = time.time() 
    Optimizer = computational_OT.semi_dual_dampedNewton_np( C,
                                                            a,
                                                            b,
                                                            f,
                                                            eps,
                                                            rho,
                                                            c,
                                                            exp_log = "True" ) 
    out = Optimizer._update( max_iterations = 50 )
    end = time.time()
    if out != -1:
        results_semi_dual_dampedNewton.append( out )
        times_semi_dual_dampedNewton.append( end - start )
        print( " |- Computing P " )
        print( "" )
        u_opt = np.exp( out['potential_f']/eps )
        K = np.exp( - C/eps )
        v_opt =  np.exp( out['potential_g']/eps )
        P_opt = GetP( u_opt, K, v_opt )
        Semi_dual_dampedNewtonP.append( P_opt )
        print( " |- Recording (unstabilized) Hessian \n " )
        mat  = - eps * Optimizer.Hessian
        diag = 1/np.sqrt( a )
        mat = diag[:, None] * mat * diag[None,:]
        Hessians_semi_dual_dampedNewton.append( mat )
    else:
        withoutprecond_epsilons.remove( eps )
# end for

#### Error plot

In [ ]:
plt.rcParams.update( { 'font.size' : 12 } )
plt.figure( figsize = ( 12, 5 ) )
plt.title( "$$" )
plt.title( "$||P1 -a||_1+||P1 -b||_1$" )
for i in range( len( results_semi_dual_dampedNewton ) ):
  error = np.asarray( results_semi_dual_dampedNewton[i]['error'] )
  plt.plot( error, label = 'Semi-dual damped Newton for $\epsilon = $' + str(withoutprecond_epsilons[i]), linewidth = 2 )
# end for
plt.xlabel( " Number of iterations " ) 
plt.ylabel( " Error in log-scale " )
plt.legend( loc = "upper right" )
plt.yscale( 'log' )
plt.tight_layout()
plt.savefig( "../Images/SemiDual_dampedNewton_images_numpy/ErrorSemi_DualNewton.pdf", format = 'pdf' )
plt.show()
print( " \n Error plots can increase! The error is not the objective function! " )

#### Objective function plot

In [ ]:
plt.rcParams.update( { 'font.size' : 12 } )
plt.figure( figsize = ( 12, 5 ) )
plt.title( "$$" )
plt.title( " Objective Function " )
for i in range( len( results_semi_dual_dampedNewton ) ):
  value = np.asarray( results_semi_dual_dampedNewton[i]['objective_values'] )
  plt.plot( value,label = 'Semi-dual damped Newton for $\epsilon = $'+ str(withoutprecond_epsilons[i]), linewidth = 2 )
# end for
plt.xlabel( " Number of iterations " )
plt.ylabel( " Objective value " )
plt.legend( loc = "upper right" )
plt.savefig( "../Images/SemiDual_dampedNewton_images_numpy/ObjectiveSemi_DualNewton.pdf", format = 'pdf' )
plt.show()

#### Plot displaying the step lengths obtained from the Armijo's conditions at different iterations for different epsilons

In [ ]:
plt.rcParams.update( { 'font.size' : 12 } )
plt.figure( figsize = ( 20, 7 ) )
plt.subplot( 2, 1, 1 )
plt.title( "Alpha" )
for i in range( len( results_semi_dual_dampedNewton ) ):
  plt.plot( np.asarray( results_semi_dual_dampedNewton[i]["linesearch_steps"] ), label = 'Damped Newton for $\epsilon = $'+ str(withoutprecond_epsilons[i]), linewidth = 2 )
# end for
plt.xlabel( "Number of iterations" )
plt.ylabel( "Alpha in log-scale" ) 
plt.legend( loc = "upper right" )
plt.savefig( "../Images/SemiDual_dampedNewton_images_numpy/AlphaSemi_DualNewton.pdf", format = 'pdf' )
plt.show()

## Observing the spectrum of the eigenvalues of the Hessian at the optimal potentials obtained from the above algorithm

In [ ]:
def print_spectral_statistics( mat, stabilize = False ):
    if stabilize:
        # Stabilizing largest and smallest eigenvalue
        min_vector = np.hstack( ( np.ones( N[0] ) ) )
        max_vector = np.hstack( ( np.ones(N[0] ) ) )
        norm = np.sqrt( N[0] )
        min_vector = min_vector/norm
        max_vector = max_vector/norm
        min_vector = min_vector.reshape( ( min_vector.shape[0], 1 ) )
        max_vector = max_vector.reshape( ( max_vector.shape[0], 1 ) )
        #
        mat = mat + np.dot( min_vector, min_vector.T )
        mat = mat - np.dot( max_vector, max_vector.T )
    # endif
    eig, v = np.linalg.eigh( mat )
    sorting_indices = np.argsort( eig )
    eig = eig[ sorting_indices ]
    v   = v[ :, sorting_indices ]
    #print( "Mean eigenvalue: ", np.mean(eig) )
    print( "List of smallest eigenvalues: ", eig[ : 10 ] )
    print( "List of largest  eigenvalues: ", eig[ - 10 : ] )
    min_index = np.argmin( eig )
    max_index = np.argmax( eig )
    min_value = eig[ min_index ]
    max_value = eig[ max_index ]
    min_vector = v[ :, min_index ]
    min_vector = min_vector/min_vector[0]
    max_vector = v[ :,max_index ]
    max_vector = max_vector/max_vector[0]
    condition_number = max_value/min_value
    # Test smallest and largest
    # print( "Min eigenvalue vector: ", min_vector)
    # print( "Max eigenvalue vector: ", max_vector)
    #
    #print( v[:,0]*np.sqrt( self.N1 + self.N2))
    #vector = v[:,0]
    #test = np.dot( result, vector)
    #print( np.linalg.norm(test) )
    #print("Min absolute eigenvalues: ", min_value)
    #print("Norm of v-1: ", np.linalg.norm(min_vector-eig_vector))
    print( " Condition number: ", condition_number )
    # plt.hist( eig, 50)
    # plt.title( "Histogram of eigenvalues for Hessian")
    # plt.xlabel( "Eigenvalues")
    # plt.yscale( "log" )
    # plt.show()
    return eig, v

In [ ]:
eigs = []
eigvecs = []
for i in range( len( withoutprecond_epsilons ) ):
    eps = withoutprecond_epsilons[i]
    print( " Spectral statistics of Hessian for epsilon = " + str(eps) )
    Hessian = Hessians_semi_dual_dampedNewton[i]
    ev = print_spectral_statistics( Hessian, stabilize = False )
    eigs.append( ev[0] )
    eigvecs.append( ev[1] )
    print("")
# end for

In [ ]:
plt.rcParams.update( { 'font.size' : 10 } )
fig, ax = plt.subplots( figsize = ( 5, 12 ), nrows =  len(withoutprecond_epsilons), ncols = 1, sharey = True )
plt.title( " Histogram of eigenvalues. " )
for i in range( len( withoutprecond_epsilons ) ):
    ax[i].hist( eigs[i], 50 )
    ax[i].set_title( " $\epsilon$: " + str(withoutprecond_epsilons[i]) )
    ax[i].set_xlabel( " Eigenvalues " )
    ax[i].set_yscale( "log" )
    ax[i].set_xlim( 0, 1 )
# end for
plt.subplots_adjust( wspace = 0, hspace = 0.5 ) 
plt.tight_layout()
plt.savefig( "../Images/DampedNewton_SemiDual_images_numpy/eigenhistunstabilized.pdf", format = 'pdf' )
plt.show()

# II. Semidual damped Newton with preconditioning
Here we perform semi-dual damped Newton with preconditioning. Here we consider $t$ eigenvalues of the Hessian that we want to move to one and form the following preconditioning matrix using the corresponding eigenvectors,
$$
P = \left(I_{n+m}-\sum_{i-1}^{t}\left(1 - \frac{1}{\sqrt{\lambda_{i}}}\right)y_{i}y_{i}^{T}\right)\ ,
$$
where
$$
y_{i} \in \ker\left(\nabla^{2}_{f}Q^{semi}_{\alpha, \beta, \varepsilon}(f)-\lambda_{i}I_{n}\right),\ \forall i= 1,\dots,k\ ,
$$
 are orthonormal.

Now, at the $k^{th}$ iteration we solve the following equation:
$$
(P\nabla^{2}Q^{semi}_{\alpha, \beta, \varepsilon}(f)P)(Pp_{k})=P\nabla Q^{semi}_{\alpha, \beta, \varepsilon}(f)\ ,
$$
using iterative inversion methods such as "Conjugate gradient" and "GMRES" to get the ascent direction $p_{k}$, following which we use the Armijo condition to obtain the ascent step size $\alpha_{k}$.

#### Preconditioning eigenvectors 
The Hessian of damped Newton in the semi-dual formulation are in the interval [0,1]. The following function collects eigenvectors corresponding to $k$ eigenvalues towards the left end of the spectrum, that is, near zero.

In [ ]:
def build_preconditioners( num_eigs, modified_Hessian, ansatz = True ):
    # Diagonalize
    eigenvalues, eigenvectors = np.linalg.eigh( modified_Hessian )
    sorting_indices = np.argsort( eigenvalues  )
    eigenvalues  = eigenvalues[ sorting_indices ]
    eigenvectors = eigenvectors[ : , sorting_indices ]
    # Form null vector
    if not ansatz:
        null_vector = eigenvectors[:, 0]
    else:
        null_vector = np.ones( N[0] ) 
        norm = np.sqrt( N[0] )
        null_vector = null_vector/norm
    # Form other vectors
    indices = []
    for i in range( num_eigs ):
        indices.append( i+1 )
    # end for
    precond_vectors = []
    for index in indices:
        precond_vectors.append( eigenvectors[ :, index ] )
    # end for
    return null_vector, precond_vectors

In [ ]:
num_eigs = 40
null_vector, precond_vectors = build_preconditioners( num_eigs, Hessians_semi_dual_dampedNewton[-1], ansatz = False )
# Cost matrix
C = distmat( x, y )
# a and b   
a = normalize( np.ones( N[0] ) )
b = normalize( np.ones( N[1] ) )
withprecond_epsilons = [ 0.5, 0.1, 0.03, 0.02 ]

## True preconditioning with exact inversion 

In [ ]:
print(  " Semi dual damped Newton with preconditioning and exact inversion... " )
print( " Doing for (",N[0], N[1],"). " )
rho = 0.6
c = 0.1
reset_starting_point = True  
final_modified_Hessians_EI = []
Semi_dual_dampedNewton_with_preconditionerP_EI = []
results_semi_dual_dampedNewton_with_preconditioner_EI = []
times_semi_dual_dampedNewton_with_preconditioner_EI = []
f = None
for eps in withprecond_epsilons :
    print( " For epsilon = "+str(eps)+":" )    
    if f is None:
        f = a * 0
    print( " |- Iterating" )  
    start = time.time() 
    Optimizer = computational_OT.semi_dual_dampedNewton_with_precodonditioning_np(  C,
                                                                                    a,
                                                                                    b,
                                                                                    f,
                                                                                    eps,
                                                                                    rho,
                                                                                    c,
                                                                                    null_vector,
                                                                                    precond_vectors[:],
                                                                                    exp_log = "True" )                                        
    out = Optimizer._update(    max_iterations = 50,
                                iterative_inversion = -1,
                                version = None,
                                debug = False,
                                optType = 'cg' )
    results_semi_dual_dampedNewton_with_preconditioner_EI.append( out )
    end = time.time()
    times_semi_dual_dampedNewton_with_preconditioner_EI.append( end - start )
    print( " |- Computing P" )
    print( "" )
    u_opt = np.exp( out['potential_f']/eps )
    K = np.exp( - C/eps )
    v_opt =  np.exp( out['potential_g']/eps )
    P_opt = GetP( u_opt, K, v_opt )
    Semi_dual_dampedNewton_with_preconditionerP_EI.append( P_opt )
    if not reset_starting_point:
        f = Optimizer.x[:a.shape[0]]
        g = Optimizer.x[a.shape[0]:]
    final_modified_Hessians_EI.append( Optimizer.modified_Hessian )
# end for

#### Error plot


In [ ]:
plt.rcParams.update( { 'font.size' : 12 } )
plt.figure( figsize = ( 20, 7 ) )   
plt.title( "$$" ) 
plt.title( "$||P1 -a||_1+||P1 -b||_1$" ) 
for i in range( len( results_semi_dual_dampedNewton_with_preconditioner_EI ) ): 
  error = np.asarray( results_semi_dual_dampedNewton_with_preconditioner_EI[i]['error'] ) 
  plt.plot( error, label = 'Semi dual damped Newton with preconditioning for $\epsilon = $'+ str(withprecond_epsilons[i]), linewidth = 2 ) 
# end for
plt.xlabel( " Number of iterations " )  
plt.ylabel( " Error in log-scale " )  
plt.legend( loc = "upper right" ) 
plt.yscale( 'log' ) 
plt.savefig( "../Images/SemiDual_dampedNewton_images_numpy/Error_Semi_DualNewton_Exact_Inversion.pdf", format = 'pdf' )  
plt.show() 
print( "\n Error plots can increase! The error is not the objective function!" ) 

#### Objective function plot

In [ ]:
plt.rcParams.update( { 'font.size' : 12 } )
plt.figure( figsize = ( 12, 5 ) )
plt.title( "$$" )
plt.title( " Objective Function " )
for i in range( len( results_semi_dual_dampedNewton_with_preconditioner_EI ) ):
  value = np.asarray( results_semi_dual_dampedNewton_with_preconditioner_EI[i]['objectives'] )
  plt.plot( value,label = 'Semi dual damped Newton with preconditioning for $\epsilon = $'+ str(withprecond_epsilons[i]), linewidth = 2 )
# end for
plt.xlabel( " Number of iterations " )
plt.ylabel( " Objective value " )
plt.legend( loc = "upper right" )
plt.savefig( "../Images/SemiDual_dampedNewton_images_numpy/ObjectiveSemi_DualNewton_Exact_Inversion.pdf", format = 'pdf' )
plt.show()

#### Ascent step-size plot

In [ ]:
plt.rcParams.update( { 'font.size' : 12 } )
plt.figure( figsize = ( 20, 7 ) )
plt.subplot( 2, 1, 1 )
plt.title( "Alpha" )
for i in range( len( results_semi_dual_dampedNewton_with_preconditioner_EI ) ):
  plt.plot( np.asarray( results_semi_dual_dampedNewton_with_preconditioner_EI[i]["linesearch_steps"] ), label = 'Semi dual damped Newton with preconditioning for $\epsilon = $'+ str(withprecond_epsilons[i]), linewidth = 2 )
# end for
plt.xlabel( "Number of iterations" )
plt.ylabel( "Alpha in log-scale" ) 
plt.legend( loc = "upper right" )
plt.savefig( "../Images/SemiDual_dampedNewton_images_numpy/AlphaSemi_DualNewton_Exact_Inversion.pdf", format = 'pdf' )
plt.show()

#### Average time plot

In [ ]:
text = [
        "Preconditioning 1: Form E data",
        "Preconditioning 2: Form P data",
        "Form preconditioning functions",
        "Invert the linear system for p_k",
        "Unwinding",
        "Complete code block"
        ]

plt.figure( figsize = ( 20, 10 ) )  
for j in range( len(results_semi_dual_dampedNewton_with_preconditioner_EI[0]['timings'][0]) ):
  values = []
  for i in range( len(results_semi_dual_dampedNewton_with_preconditioner_EI) ):
    mean = 0
    for k in range( len(results_semi_dual_dampedNewton_with_preconditioner_EI[i]['timings']) ):
      mean += results_semi_dual_dampedNewton_with_preconditioner_EI[i]['timings'][k][j]
    # end for
    mean = mean/len( results_semi_dual_dampedNewton_with_preconditioner_EI[i]['timings'] ) 
    values.append( mean )
  # end for
  if len(withprecond_epsilons) == len(values):
    plt.plot( withprecond_epsilons, np.asarray(values), label = text[j], linewidth = 2 )
    plt.legend( loc = "upper right" )
# end for
plt.xlabel( "$\epsilon$" )
plt.ylabel( "Time in ms" )
plt.savefig( "../Images/SemiDual_dampedNewton_images_numpy/Timings_Semi_DualNewton_Exact_Inversion.pdf", format = 'pdf' )
plt.show()

## True preconditioning with iterative inversion

In [ ]:
print(  " Semi dual damped Newton with preconditioning and iterative inversion... "  )
print( " Doing for (",N[0], N[1],"). " )
rho = 0.6
c = 0.1
reset_starting_point = True  
final_modified_Hessians_I_inv = []
Semi_dual_dampedNewton_with_preconditionerP_I_inv = []
results_semi_dual_dampedNewton_with_preconditioner_I_inv = []
times_semi_dual_dampedNewton_with_preconditioner_I_inv = []
iter_inv = 20
r_tol = 1e-5 
a_tol = 1e-12
f = None
for eps in withprecond_epsilons :
    print( " For epsilon = "+str(eps)+":" )    
    if f is None:
        f = a * 0
    print( " |- Iterating" )  
    start =time.time() 
    Optimizer = computational_OT.semi_dual_dampedNewton_with_precodonditioning_np(  C,
                                                                                    a,
                                                                                    b,
                                                                                    f,
                                                                                    eps,
                                                                                    rho,
                                                                                    c,
                                                                                    null_vector,
                                                                                    precond_vectors[:],
                                                                                    exp_log = "True" )
    out = Optimizer._update(    max_iterations = 50,
                                iterative_inversion = iter_inv,
                                relative_tol = r_tol,
                                absolute_tol = a_tol,
                                version = None,
                                debug = False,
                                optType = 'cg' )
    results_semi_dual_dampedNewton_with_preconditioner_I_inv.append( out )
    end = time.time()
    times_semi_dual_dampedNewton_with_preconditioner_I_inv.append( end - start )
    print( " |- Computing P" )
    print( "" )
    u_opt = np.exp( out['potential_f']/eps )
    K = np.exp( - C/eps )
    v_opt =  np.exp( out['potential_g']/eps )
    P_opt = GetP( u_opt, K, v_opt )
    Semi_dual_dampedNewton_with_preconditionerP_I_inv.append( P_opt )
    if not reset_starting_point:                   
        f = Optimizer.x[:a.shape[0]]
        g = Optimizer.x[a.shape[0]:]
    final_modified_Hessians_I_inv.append( Optimizer.modified_Hessian )
# end for

#### Error plot

In [ ]:
plt.rcParams.update( { 'font.size' : 12 } )
plt.figure( figsize  = ( 20, 7 ) )   
plt.title( "$$" ) 
plt.title( "$||P1 -a||_1+||P1 -b||_1$" ) 
for i in range( len( results_semi_dual_dampedNewton_with_preconditioner_I_inv ) ): 
  error = np.asarray( results_semi_dual_dampedNewton_with_preconditioner_I_inv[i]['error'] ) 
  plt.plot( error, label = 'Semi dual damped Newton with preconditioning for $\epsilon = $'+ str(withprecond_epsilons[i]), linewidth = 2 ) 
# end for
plt.xlabel( " Number of iterations " )  
plt.ylabel( " Error in log-scale " )  
plt.legend( loc = "upper right" ) 
plt.yscale( 'log' ) 
plt.savefig( "../Images/SemiDual_dampedNewton_images_numpy/Error_Semi_DualNewton_CG.pdf", format = 'pdf' ) 
plt.show() 
print( "\n Error plots can increase! The error is not the objective function!" ) 

#### Objective function plot

In [ ]:
plt.rcParams.update( { 'font.size' : 12 } )
plt.figure( figsize = ( 12, 5 ) )
plt.title( "$$" )
plt.title( " Objective Function " )
for i in range( len( results_semi_dual_dampedNewton_with_preconditioner_I_inv ) ):
  value = np.asarray( results_semi_dual_dampedNewton_with_preconditioner_I_inv[i]['objectives'] )
  plt.plot( value,label = 'Semi dual damped Newton with preconditioning for $\epsilon = $'+ str(withprecond_epsilons[i]), linewidth = 2 )
# end for
plt.xlabel( " Number of iterations " )
plt.ylabel( " Objective value " )
plt.legend( loc = "upper right" )
plt.savefig( "../Images/SemiDual_dampedNewton_images_numpy/ObjectiveSemi_DualNewton_CG.pdf", format = 'pdf' )
plt.show()

#### Ascent step-size plot

In [ ]:
plt.rcParams.update( { 'font.size' : 12 } )
plt.figure( figsize = ( 20, 7 ) )
plt.subplot( 2, 1, 1 )
plt.title( "Alpha" )
for i in range( len( results_semi_dual_dampedNewton_with_preconditioner_I_inv ) ):
  plt.plot( np.asarray( results_semi_dual_dampedNewton_with_preconditioner_I_inv[i]["linesearch_steps"] ), label = 'Semi dual damped Newton with preconditioning for $\epsilon = $'+ str(withprecond_epsilons[i]), linewidth = 2 )
# end for
plt.xlabel( "Number of iterations" )
plt.ylabel( "Alpha in log-scale" ) 
plt.legend( loc = "upper right" )
plt.savefig( "../Images/SemiDual_dampedNewton_images_numpy/AlphaSemi_DualNewton_CG.pdf", format = 'pdf' )
plt.show()

#### Average time plot

In [ ]:
plt.rcParams.update( { 'font.size' : 12 } )
text = [
        "Preconditioning 1: Form E data",
        "Preconditioning 2: Form P data",
        "Form preconditioning functions",
        "Invert the linear system for p_k",
        "Unwinding",
        "Complete code block"
        ]

plt.figure( figsize = ( 20, 10 ) )  
for j in range( len(results_semi_dual_dampedNewton_with_preconditioner_I_inv[0]['timings'][0]) ):
  values = []
  for i in range( len(results_semi_dual_dampedNewton_with_preconditioner_I_inv) ):
    mean = 0
    for k in range( len(results_semi_dual_dampedNewton_with_preconditioner_I_inv[i]['timings']) ):
      mean += results_semi_dual_dampedNewton_with_preconditioner_I_inv[i]['timings'][k][j]
    # end for
    mean = mean/len( results_semi_dual_dampedNewton_with_preconditioner_I_inv[i]['timings'] ) 
    values.append( mean )
  # end for
  if len(withprecond_epsilons) == len(values):
    plt.plot( withprecond_epsilons, np.asarray(values), label = text[j], linewidth = 2 )
    plt.legend( loc = "upper right" )
# end for
plt.xlabel( "$\epsilon$" )
plt.ylabel( "Time in ms" )
plt.savefig( "../Images/SemiDual_dampedNewton_images_numpy/Timings_Semi_DualNewton_CG.pdf", format = 'pdf' )
plt.show()

## Time plot to compare exact inversion and iterative inversion convergence of semi dual damped Newton

In [ ]:
plt.rcParams.update( { 'font.size' : 12 } )                                                                                                                          
plt.figure( figsize = ( 20, 7 ) )   
plt.title( "$$" ) 
plt.title( "Time plot" ) 
plt.plot( times_semi_dual_dampedNewton_with_preconditioner_EI[::-1], label = 'Semi dual damped Newton with true preconditioning and exact inversion with '+str(num_eigs)+ ' preconditioning vectors', linewidth = 2, marker = 'o' ) 
plt.plot( times_semi_dual_dampedNewton_with_preconditioner_I_inv[::-1], label = 'Semi dual damped Newton with true preconditioning and CG with (num_precond, iter_inv, atol, rtol) = ('+str(num_eigs)+', '+str(iter_inv)+', '+str(a_tol)+',  '+str(r_tol)+') ', linewidth = 2, marker = 'o' ) 
plt.xticks( np.arange( len( withprecond_epsilons ) ), withprecond_epsilons[::-1] ) 
plt.xlabel( "$\epsilon$" )  
plt.ylabel( "Time in sec" ) 
plt.legend( loc = "upper right" )
plt.savefig( "../Images/SemiDual_dampedNewton_images_numpy/Timeplot_EI_vs_II.pdf", format = 'pdf' ) 
plt.show()                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                        